<a href="https://colab.research.google.com/github/Athul-nambiar/Fine-Tuning-/blob/main/FineTuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers accelerate bitsandbytes


In [2]:
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_name = "aboonaji/llama2finetune-v2"

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Corrected BitsAndBytesConfig for 8-bit quantization
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,  # Use 8-bit quantization
    llm_int8_threshold=6.0,
    llm_int8_enable_fp32_cpu_offload=True  # ✅ Correct setting for CPU offloading
)

# Corrected Model Loading with Custom Device Map
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"  # ✅ Auto-distributes model across CPU/GPU
)

print("✅ Model loaded successfully!")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

✅ Model loaded successfully!


In [38]:
import torch

# Move model to GPU if available
device = "cuda" if torch.cuda.is_available() else "cpu"


# Define your prompt
prompt = "can you give me about stone man syndrome?"

# Tokenize the input
inputs = tokenizer(prompt, return_tensors="pt").to(device)

# Generate response
with torch.no_grad():
    output = model.generate(
        **inputs,
        max_new_tokens=250,  # Adjust as needed
        temperature=0.5,  # Controls randomness (lower = more focused)
        top_p=1,  # Nucleus sampling (higher = more diverse output)
        repetition_penalty=1.1,  # Reduce repetitive text
    )

# Decode and print result
response = tokenizer.decode(output[0], skip_special_tokens=True)
print(response)


NotImplementedError: Cannot copy out of meta tensor; no data!

In [ ]:
from datasets import load_dataset

# Load the dataset
ds = load_dataset("aboonaji/wiki_medical_terms_llam2_format")



In [ ]:
# Check available splits
print(ds.keys())

# Access the training split
train_dataset = ds["train"]


dict_keys(['train'])


In [ ]:
from sklearn.model_selection import train_test_split
from datasets import Dataset

# Access the 'train' split from the dataset
train_data = ds["train"]

# Convert to pandas DataFrame for easier manipulation
train_df = train_data.to_pandas()

# First split into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Now split the train_df into train and validation sets (80% train, 20% validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Convert back to datasets.Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Check the splits
print(f"Train dataset size: {len(train_dataset)}")
print(f"Validation dataset size: {len(val_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")


Train dataset size: 5488
Test dataset size: 1373


In [4]:
from transformers import AutoTokenizer
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split

# Load the dataset
train_data = ds["train"]

# Convert to pandas DataFrame for easier manipulation
train_df = train_data.to_pandas()

# First split into train and test sets (80% train, 20% test)
train_df, test_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Now split the train_df into train and validation sets (80% train, 20% validation)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

# Convert back to datasets.Dataset
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_df)

# Load the tokenizer (choose the one based on your model)
tokenizer = AutoTokenizer.from_pretrained('aboonaji/llama2finetune-v2')

# Define the preprocess function for tokenization
def preprocess_function(examples):
    # Assuming the 'text' field contains the raw input text
    return tokenizer(examples['text'], truncation=True, padding="max_length", max_length=512)

# Apply preprocessing to the training dataset
train_tokenized_ds = train_dataset.map(preprocess_function, batched=True)

# Apply preprocessing to the validation dataset
val_tokenized_ds = val_dataset.map(preprocess_function, batched=True)

# Apply preprocessing to the test dataset
test_tokenized_ds = test_dataset.map(preprocess_function, batched=True)

# Check a sample from the tokenized training dataset to verify
print(train_tokenized_ds[0])  # Prints the first tokenized example from training data

# Optionally, check a sample from the validation dataset
print(val_tokenized_ds[0])  # Prints the first tokenized example from validation data

# Optionally, check a sample from the test dataset
print(test_tokenized_ds[0])  # Prints the first tokenized example from test data


NameError: name 'train_dataset' is not defined

{'text': '<s> [INST] <<SYS>> You are a helpful, respectful, and honest assistant. Always answer as helpfully as possible, while being safe. Your answers should not include any harmful, unethical, racist, sexist, toxic, dangerous, or illegal content. Please ensure that your responses are socially unbiased and positive in nature. If a question does not make any sense or is not factually coherent, explain why instead of answering something not correct. If you don\'t know the answer to a question, please don\'t share false information. <</SYS>> What is Paracetamol poisoning and explain in detail? [/INST] Paracetamol poisoning, also known as acetaminophen poisoning, is caused by excessive use of the medication paracetamol (acetaminophen). Most people have few or non-specific symptoms in the first 24 hours following overdose. These include feeling tired, abdominal pain, or nausea. This is typically followed by a couple of days without any symptoms, after which yellowish skin, blood clotting 

❌ LoRA is NOT applied. You need to wrap the model with LoRA first.
